In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import os
from copy import copy

import numpy as np
import pandas as pd

from utils.generic_utils import pickle_in,pickle_out
import utils.preprocessing_utils as prep
from utils.evaluation_utils import rmsle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
kaggle_folder = os.path.join("../","datasets","kaggle_sets")
kaggle_test = pd.read_csv(os.path.join(kaggle_folder,"test.csv"))

initial_data_folder = os.path.join("../","datasets","initial_data_split")
initial_data_test = pd.read_csv(os.path.join(initial_data_folder,"test.csv"))

In [5]:
print initial_data_test.head()

              datetime  season  holiday  workingday  weather   temp   atemp  \
0  2011-01-16 00:00:00       1        0           0        1  10.66  15.150   
1  2011-01-16 01:00:00       1        0           0        1  10.66  13.635   
2  2011-01-16 02:00:00       1        0           0        1  10.66  14.395   
3  2011-01-16 03:00:00       1        0           0        1   9.02  13.635   
4  2011-01-16 04:00:00       1        0           0        1  10.66  12.880   

   humidity  windspeed  casual  registered  count  
0        56     0.0000       6          33     39  
1        56     8.9981       4          19     23  
2        56     6.0032       3          13     16  
3        69     0.0000       9           6     15  
4        56    11.0014       0           1      1  


In [6]:
X_test = kaggle_test
kaggle_datetime = kaggle_test[["datetime"]]

X_test_my = copy(initial_data_test).drop(["count","registered","casual"], axis=1)
Y_test_my = copy(initial_data_test)[["count"]].values.ravel()

full_pipeline = pickle_in(os.path.join("../","models","random_forest_weekdays_pipeline.pkl"),compresion_mode=5)

Y_kaggle = full_pipeline.predict(X_test)
Y_test_my_pred = full_pipeline.predict(X_test_my)

test_error = rmsle(Y_test_my_pred,Y_test_my)
print "Score on my testset: %s\n"%test_error

kaggle_datetime["count"] = Y_kaggle

kaggle_datetime.to_csv(os.path.join("../","submissions",
                                    "random_forest_weekdays_submission.csv"),index=False)

print kaggle_datetime.head()

Score on my testset: 0.357134687056

              datetime      count
0  2011-01-20 00:00:00  10.807449
1  2011-01-20 01:00:00   5.512882
2  2011-01-20 02:00:00   3.399814
3  2011-01-20 03:00:00   3.356646
4  2011-01-20 04:00:00   3.045574


/home/jakubczakon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
